In [174]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'simple_white'

In [214]:
df = pd.read_csv('new_rumah 123.csv')
df.head()

,km-tidur,km-mandi,garasi,bangunan,tanah,lokasi,umr,jlh-pdk,harga
0,22,22,1,218,92,Jakarta,4641854,11249585,2.700000e+09
1,20,14,0,231,92,Jakarta,4641855,11249586,2.700000e+09
2,20,14,7,1100,746,Jakarta,4641856,11249587,3.500000e+10
3,20,14,2,350,300,Jakarta,4641857,11249588,8.500000e+09
4,18,6,1,450,300,Jakarta,4641858,11249589,1.650000e+10


In [215]:
df['harga'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   km-tidur  5982 non-null   int64  
 1   km-mandi  5982 non-null   int64  
 2   garasi    5982 non-null   int64  
 3   bangunan  5982 non-null   int64  
 4   tanah     5982 non-null   int64  
 5   lokasi    5982 non-null   object 
 6   umr       5982 non-null   int64  
 7   jlh-pdk   5982 non-null   int64  
 8   harga     5982 non-null   float64
dtypes: float64(1), int64(7), object(1)
memory usage: 420.7+ KB


In [177]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
km-tidur,5982.0,3.277834e+00,1.289952e+00,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,2.200000e+01
km-mandi,5982.0,2.560348e+00,1.320407e+00,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,2.200000e+01
garasi,5982.0,9.598796e-01,1.086493e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.500000e+01
bangunan,5982.0,1.775376e+02,2.098867e+02,1.000000e+00,6.800000e+01,1.150000e+02,2.000000e+02,5.000000e+03
tanah,5982.0,1.772539e+02,2.444716e+02,1.000000e+00,7.500000e+01,1.050000e+02,1.800000e+02,4.758000e+03
umr,5982.0,2.606652e+07,2.045624e+07,4.280214e+06,4.330758e+06,4.280118e+07,4.641119e+07,4.791184e+07
jlh-pdk,5982.0,2.309005e+07,3.171812e+07,1.376734e+06,3.245674e+06,1.124998e+07,2.179174e+07,1.124916e+08
harga,5982.0,4.744525e+09,9.474065e+09,1.000000e+09,1.720000e+09,2.993723e+09,4.881466e+09,2.700000e+11


In [178]:
df.isnull().sum()

km-tidur    0
km-mandi    0
garasi      0
bangunan    0
tanah       0
lokasi      0
umr         0
jlh-pdk     0
harga       0
dtype: int64

In [179]:
df.drop_duplicates(inplace=True)

In [180]:
cp_df = df.copy()
for x in cp_df['tanah']:
    if x <= 100:
        cp_df.tanah.replace(x, '<=100', inplace=True)
    elif x > 100 and x <=200:
        cp_df.tanah.replace(x, '<=200', inplace=True)
    elif x > 200 and x <= 300:
        cp_df.tanah.replace(x, '<=300', inplace=True)
    else:
        cp_df.tanah.replace(x, '>400', inplace=True)


In [212]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'xy'}, {'type':'domain'}]])

fig.add_trace(go.Histogram(x = cp_df["tanah"],name="Luas Tanah"),row=1, col=1)
fig.add_trace(go.Pie(labels = cp_df["tanah"] ,name="Luas Tanah", textinfo='percent+label', textposition='inside', marker=dict(line=dict(color='white', width=1))),row=1, col=2)

fig.update_layout(showlegend=False, title="Luas Tanah (m²)", title_x=.5)

fig.show()

In [182]:
cp_df['TM'] = cp_df['km-tidur'] + cp_df['km-mandi']
fig = make_subplots(rows=2, cols=2, specs=[[{'type':'xy'}, {'type':'xy'}],[{'type':'xy'}, None]], subplot_titles=("Jumlah Kamar Tidur, Kamar mandi", "Jumlah Kamar Tidur", "Jumlah Kamar Mandi"))

fig.add_trace(go.Histogram(x = cp_df["TM"],name="TM"),row=1, col=1)
fig.add_trace(go.Histogram(x = df["km-tidur"] ,name="Kamar Tidur"),row=2, col=1)
fig.add_trace(go.Histogram(x = df["km-mandi"] ,name="Kamar Mandi"),row=1, col=2)

fig.update_layout(showlegend=False, title_x=.5)

fig.show()

In [183]:
fig = px.scatter(df, x='tanah', y='harga', color="lokasi",  trendline='ols')
fig.update_layout(
    title={
        'text': "Luas Tanah VS Harga",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [184]:
fig = px.scatter(df, x="bangunan", y="harga", color="lokasi", trendline='ols')
fig.update_layout(
    title={
        'text': "Luas Bangunan VS Harga",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [185]:
df['km-tidur'] = df['km-tidur'].astype(int)

In [186]:
cp_df2 = df.copy()
for x in cp_df2['lokasi']:
    if x == "Jakarta":
        cp_df2.lokasi.replace(x, 1, inplace=True)
    elif x == "Bogor":
        cp_df2.lokasi.replace(x, 2, inplace=True)
    elif x == "Depok":
        cp_df2.lokasi.replace(x, 3, inplace=True)
    elif x == "Tangerang":
        cp_df2.lokasi.replace(x, 4, inplace=True)
    elif x == "Tangerang Selatan":
        cp_df2.lokasi.replace(x, 5, inplace=True)
    elif x == "Bekasi":
        cp_df2.lokasi.replace(x, 6, inplace=True)

In [187]:
df_corr = cp_df2.corr()

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = df_corr.columns,
        y = df_corr.index,
        z = np.array(df_corr),
        text=df_corr.values,
        texttemplate='%{text:.2f}',
        colorscale = ["#1f76b5", "white", "#ff7e0f"],
        showscale=False
    )
)
fig.update_layout(height=500, title="Correlation Matrix", title_x=.5)
fig.show()

In [188]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['lokasi']= le.fit_transform(df['lokasi'])
df['lokasi'].unique()

array([3, 1, 2, 4, 5, 0])

In [189]:
from sklearn.preprocessing import StandardScaler
y = df[["harga"]]
X = df.drop("harga", axis = 1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

In [190]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, random_state=2)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(4785, 8) (1197, 8)
(4785, 1) (1197, 1)


In [191]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(x_train, np.ravel(y_train, order='C'))

RandomForestRegressor(random_state=0)

In [192]:
rf_pred = rf.predict(x_test)
rf_pred

array([-0.29732332,  0.27334535, -0.00760296, ...,  0.69073014,
       -0.30375889, -0.07208502])

In [193]:
from sklearn.metrics import r2_score
r2_score(y_test, rf_pred)

0.7130998373579609

In [194]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    "n_estimators":[100,200,300],
    "max_depth":[10, 50, 100],
    "max_features":[6,8,10,12,14,16],
    "min_samples_split" : [2, 6, 10],
    "min_samples_leaf" : [1, 3, 4],
    "bootstrap" : [True, False]
}

rf_reg = RandomForestRegressor()

rf_reg_tuned = GridSearchCV(estimator=rf_reg,
                            param_grid=param_grid,
                            cv=3,
                            n_jobs=-1,
                            verbose=2)


# n_estimators = [5,20,50,100, 200, 300, 400] # number of trees in the random forest
# max_features = ['auto', 'sqrt'] # number of features in consideration at every split
# max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
# min_samples_split = [2, 6, 10] # minimum sample number to split a node
# min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
# bootstrap = [True, False] # method used to sample data points

# random_grid = {'n_estimators': n_estimators,

# 'max_features': max_features,

# 'max_depth': max_depth,

# 'min_samples_split': min_samples_split,

# 'min_samples_leaf': min_samples_leaf,

# 'bootstrap': bootstrap}

In [195]:
rf_reg_tuned.fit(x_train, np.ravel(y_train, order='C'))
rf_reg_tuned.best_estimator_

# from sklearn.model_selection import RandomizedSearchCV
# rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
#                n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

Fitting 3 folds for each of 972 candidates, totalling 2916 fits


RandomForestRegressor(max_depth=100, max_features=14, n_estimators=200)

In [206]:
randmf = RandomForestRegressor(n_estimators = 200, min_samples_split = 10, min_samples_leaf= 4, max_features = 14, max_depth= 100, bootstrap=True) 
randmf.fit( x_train, y_train) 

C:\Users\yerem\AppData\Local\Temp\ipykernel_19136\704643953.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(max_depth=100, max_features=14, min_samples_leaf=4,
                      min_samples_split=10, n_estimators=200)

In [207]:
rf_pred = randmf.predict(x_test)
rf_pred

array([-0.22755615,  0.10195955, -0.06343219, ...,  0.52597046,
       -0.30156348, -0.06517712])

In [208]:
from sklearn.metrics import r2_score
r2_score(y_test, rf_pred)

0.739798012185769